<a href="https://colab.research.google.com/github/melikesahn/image-classification/blob/main/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/MyDrive/ilac15

/content/gdrive/MyDrive/ilac15


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14228875552867471568
 xla_global_id: -1]

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential

from keras.layers import Conv2D, MaxPooling2D

from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

#giriş verileri boyutları
img_width ,img_height=224, 224
train_data_yolu='/content/gdrive/MyDrive/ilac15/train'
validation_data_yolu='/content/gdrive/MyDrive/ilac15/validation'
train_ornek_sayisi=492
validation_ornek_sayisi=117
epochs=20
batch_size=1 #her seferinde eğitim için alınacak veri miktarı
#train_ornek sayisi,validation_ornek_sayisi batch_size ile tam bölünmeli
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


In [ ]:
#model oluşturma
model=Sequential()
#giriiş katmanı ve conv1
model.add(Conv2D(32,(3,3),input_shape=input_shape)) #32 filtre sayısı 3x3 konvulsuyon boyutu
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))        # 2x2 moxpool uygulanmış yani resim boyutu yarıya indiriliyor
#conv2
model.add(Conv2D(32,(5,5)))
model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#conv3
model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#flatten düzleştirme
model.add(Flatten())
#Dense=FC oluşturuluyor
model.add(Dense(64))  #64 nöron var
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(15)) # 15 class var
model.add(Activation('sigmoid'))
# model derleme
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])




In [ ]:
#train işlemi için kullanılan verileri çoğaltıyor
train_datalar=ImageDataGenerator(
    rescale=1. /255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest',
    horizontal_flip=True

)
#test iişelemi için görüntüleri rescle boyutlandırma
test_datalar=ImageDataGenerator(rescale=1. /255)

train_generator=train_datalar.flow_from_directory(
    train_data_yolu,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode='categorical'
)


validation_generator = test_datalar.flow_from_directory(
    validation_data_yolu,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


Found 492 images belonging to 15 classes.
Found 117 images belonging to 15 classes.


In [ ]:
#modelin eğitimi
model.fit_generator(
    train_generator,
    steps_per_epoch=train_ornek_sayisi // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_ornek_sayisi // batch_size

)

<ipython-input-16-5f1d5ed06048>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/20
492/492 [==============================] - 365s 739ms/step - loss: 2.8504 - accuracy: 0.0813 - val_loss: 2.5620 - val_accuracy: 0.1026
Epoch 2/20
492/492 [==============================] - 306s 621ms/step - loss: 2.6384 - accuracy: 0.1626 - val_loss: 2.4642 - val_accuracy: 0.1880
Epoch 3/20
492/492 [==============================] - 306s 620ms/step - loss: 2.5200 - accuracy: 0.1768 - val_loss: 2.3745 - val_accuracy: 0.2735
Epoch 4/20
492/492 [==============================] - 305s 619ms/step - loss: 2.4338 - accuracy: 0.2541 - val_loss: 2.5017 - val_accuracy: 0.3077
Epoch 5/20
492/492 [==============================] - 300s 610ms/step - loss: 2.2304 - accuracy: 0.3211 - val_loss: 5.3647 - val_accuracy: 0.2735
Epoch 6/20
492/492 [==============================] - 300s 609ms/step - loss: 2.3136 - accuracy: 0.3415 - val_loss: 3.5290 - val_accuracy: 0.2821
Epoch 7/20
492/492 [==============================] - 305s 618ms/step - loss: 2.2224 - accuracy: 0.3435 - val_loss: 1.9909 -

In [ ]:
model.save_weights('/content/gdrive/MyDrive/ilac15/model.h5')

In [ ]:
model.save('/content/gdrive/MyDrive/ilac15/model_tam.h5')

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

model.load_weights('model.h5')
img=load_img('/content/gdrive/MyDrive/deneme 2.2.1/resim2.jpg',grayscale=False, color_mode='rgb',target_size=(img_width,img_height))
x=img_to_array(img)
prediction=model.predict(x.reshape((1,img_width,img_height,3)),batch_size=32,verbose=0)  #(1,,224,224,3)
print(prediction)


[[0.10322837 0.0863697  0.11476606 0.09810972 0.12894934 0.12148099
  0.11487137 0.07316046 0.0638599  0.09520409]]


In [ ]:
!pip install tensorflow
  # Eğer GPU desteği kullanmak istiyorsanız
!pip install tflite


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 4.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import tensorflow as tf

# Kaynak ve hedef dosya yollarını belirtin
source_model_path = '/content/gdrive/MyDrive/ilac15/model_tam.h5'
tflite_model_path = '/content/gdrive/MyDrive/ilac15/model.tflite'

# TensorFlow Keras modelini yükleyin
model = tf.keras.models.load_model(source_model_path)

# TensorFlow Lite Converter'ı kullanarak dönüştürme yapın
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Dönüştürülen modeli .tflite dosyasına kaydedin
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)
